#### 🟩 Inicialização do "Compute"

In [0]:
# Instala bibliotecas necessárias do python para realizar a execução dos notebooks
# Nessa versão community que estamos utilizando, é neccessário instalar toda vez que cria um compute. 🥲 
# tentei automatizar isso, mas não consegui. 

%pip install openpyxl kaggle pandas numpy
# Em qualquer notebook Databricks

Python interpreter will be restarted.
Python interpreter will be restarted.


#### Coleta: 
🟩 Importação de dados (_RAW DATA, Bronze Stage_)

In [0]:
# Importa os metadados do projeto e gera um dataframe pra eventuais manipulações
import os
import pandas as pd

# Caminho original no DBFS
RAW_DIR = "dbfs:/FileStore/tables/"
METADADOS_FILE = RAW_DIR + "metadata.xlsx"
# Caminho temporário para o arquivo ser acessado pelo Pandas
TEMP_FILE = "/tmp/metadata.xlsx"
# Copiar arquivo do DBFS para o diretório temporário
dbutils.fs.cp(METADADOS_FILE, "file:" + TEMP_FILE, True)
# Carregar a planilha ignorando as duas primeiras linhas
METADADOS = pd.read_excel(TEMP_FILE, skiprows=2, engine="openpyxl")
# Exibir as primeiras linhas do DataFrame
METADADOS.head()

,Column Name,Data Type,Description
0,Student_ID,String,Unique identifier for each student
1,First_Name,String,Student’s first name
2,Last_Name,String,Student’s last name
3,Email,String,Contact email (can be anonymized)
4,Gender,Categorical,"Male, Female, Other"


In [0]:
# Realiza o estágio BRONZE, copiando o arquivo e salvando em CSV

# Caminho do arquivo original no DBFS
STUDENTS_FILE_DBFS = "dbfs:/FileStore/tables/Students_Grading_Dataset.csv"
STUDENTS_FILE_LOCAL = "/tmp/Students_Grading_Dataset.csv"

# Copiar do DBFS para um caminho local
dbutils.fs.cp(STUDENTS_FILE_DBFS, "file:" + STUDENTS_FILE_LOCAL, True)

# Verificar se o arquivo foi copiado corretamente
if not os.path.exists(STUDENTS_FILE_LOCAL):
    raise FileNotFoundError(f"Arquivo {STUDENTS_FILE_LOCAL} não encontrado.")

# Carregar com Pandas
BRONZE = pd.read_csv(STUDENTS_FILE_LOCAL)

# Exibir as primeiras linhas
print("Estrutura do DataFrame BRONZE:")
print(BRONZE.head())

# Criar diretório no DBFS para salvar o CSV
BRONZE_DIR_DBFS = "dbfs:/FileStore/tables/1_BRONZE"
dbutils.fs.mkdirs(BRONZE_DIR_DBFS)

# Salvar o arquivo temporariamente em /tmp
BRONZE_TEMP_PATH = "/tmp/BRONZE.csv"
BRONZE.to_csv(BRONZE_TEMP_PATH, decimal='.', index=False, encoding='utf-8')

# Copiar o CSV local (/tmp) para o DBFS
BRONZE_FILE_DBFS = "dbfs:/FileStore/tables/1_BRONZE/BRONZE.csv"
dbutils.fs.cp("file:" + BRONZE_TEMP_PATH, BRONZE_FILE_DBFS)

# Confirmação
print(f"Arquivo BRONZE salvo corretamente em: {BRONZE_FILE_DBFS}")


Estrutura do DataFrame BRONZE:
  Student_ID First_Name Last_Name                    Email  Gender  Age  \
0      S1000       Omar  Williams  student0@university.com  Female   22   
1      S1001      Maria     Brown  student1@university.com    Male   18   
2      S1002      Ahmed     Jones  student2@university.com    Male   24   
3      S1003       Omar  Williams  student3@university.com  Female   24   
4      S1004       John     Smith  student4@university.com  Female   23   

    Department  Attendance (%)  Midterm_Score  Final_Score  ...  \
0  Engineering           52.29          55.03        57.82  ...   
1  Engineering           97.27          97.23        45.80  ...   
2     Business           57.19          67.05        93.68  ...   
3  Mathematics           95.15          47.79        80.63  ...   
4           CS           54.18          46.59        78.89  ...   

   Projects_Score  Total_Score  Grade  Study_Hours_per_Week  \
0           85.90        56.09      F               

#### Modelagem: 
⭐ Esquema em Estrela (Star Schema)

In [0]:
# Cria a modelagem do Estrela (Star Schema) do Bronze, gerando os arquivos prata.

# Importar PySpark e funções
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, monotonically_increasing_id

# Criar a sessão do Spark (se não tiver sido criada ainda)
spark = SparkSession.builder.appName("Modelagem Dimensional").getOrCreate()

# Carregar o arquivo CSV com PySpark (caso o BRONZE não seja um DataFrame PySpark)
BRONZE = spark.read.option("header", "true").option("inferSchema", "true").csv("dbfs:/FileStore/tables/1_BRONZE/BRONZE.csv")

# Criar dimensão ALUNOS
ALUNOS = (BRONZE
    .select(
        col("Student_ID").alias("ID_Aluno"),
        "First_Name", "Last_Name", "Email", "Gender", "Age",
        "Parent_Education_Level", "Family_Income_Level"
    )
)

# Criar dimensão DISCIPLINAS (sem repetição, com ID incremental)
DISCIPLINAS = (BRONZE
    .select("Department")
    .distinct()
    .withColumnRenamed("Department", "Nome_Disciplina")
    .withColumn("ID_Disciplina", monotonically_increasing_id())
)

# Criar dimensão HÁBITOS
HABITOS = (BRONZE
    .select(
        col("Student_ID").alias("ID_Aluno"),
        "Study_Hours_per_Week",
        "Extracurricular_Activities",
        "Internet_Access_at_Home",
        col("Stress_Level (1-10)"),
        "Sleep_Hours_per_Night"
    )
)

# Criar tabela fato DESEMPENHO
FATO_DESEMPENHO = (BRONZE
    .select(
        col("Student_ID").alias("ID_Aluno"),
        col("Department").alias("ID_Disciplina"),
        col("Attendance (%)").alias("Presenca"),
        col("Midterm_Score").alias("Nota_Midterm"),
        col("Final_Score").alias("Nota_Final"),
        col("Projects_Score").alias("Nota_Projetos"),
        col("Total_Score").alias("Nota_Total"),
        col("Grade").alias("Conceito")
    )
)

# Definir o diretório PRATA no DBFS
PRATA_DIR_DBFS = "dbfs:/FileStore/tables/2_PRATA"

# Criar o diretório no DBFS
dbutils.fs.mkdirs(PRATA_DIR_DBFS)

# Salvar as tabelas no diretório PRATA como CSV, sobrescrevendo se já existirem

# Dimensão ALUNOS
ALUNOS.write.mode("overwrite").option("header", "true").csv(f"{PRATA_DIR_DBFS}/ALUNOS.csv")

# Dimensão DISCIPLINAS
DISCIPLINAS.write.mode("overwrite").option("header", "true").csv(f"{PRATA_DIR_DBFS}/DISCIPLINAS.csv")

# Dimensão HÁBITOS
HABITOS.write.mode("overwrite").option("header", "true").csv(f"{PRATA_DIR_DBFS}/HABITOS.csv")

# Tabela FATO_DESEMPENHO
FATO_DESEMPENHO.write.mode("overwrite").option("header", "true").csv(f"{PRATA_DIR_DBFS}/FATO_DESEMPENHO.csv")

print("Arquivos sobrescritos com sucesso em:", PRATA_DIR_DBFS)



Arquivos sobrescritos com sucesso em: dbfs:/FileStore/tables/2_PRATA


In [0]:
# Gera as views temporárias do csv para visualizar no Databricks.

# ALUNOS
spark.read.option("header", "true").csv(f"{PRATA_DIR_DBFS}/ALUNOS.csv").createOrReplaceTempView("csv_alunos")

# DISCIPLINAS
spark.read.option("header", "true").csv(f"{PRATA_DIR_DBFS}/DISCIPLINAS.csv").createOrReplaceTempView("csv_disciplinas")

# HÁBITOS
spark.read.option("header", "true").csv(f"{PRATA_DIR_DBFS}/HABITOS.csv").createOrReplaceTempView("csv_habitos")

# FATO_DESEMPENHO
spark.read.option("header", "true").csv(f"{PRATA_DIR_DBFS}/FATO_DESEMPENHO.csv").createOrReplaceTempView("csv_fato_desempenho")


#### Carga:
⬆️ Vizualização e carregamento de dados (_Loading_)

###### _🗃️_ Tabela Alunos

In [0]:
%sql
-- Exemplo de dados --
SELECT * FROM csv_alunos LIMIT 10;

ID_Aluno,First_Name,Last_Name,Email,Gender,Age,Parent_Education_Level,Family_Income_Level
S1000,Omar,Williams,student0@university.com,Female,22,High School,Medium
S1001,Maria,Brown,student1@university.com,Male,18,None,Medium
S1002,Ahmed,Jones,student2@university.com,Male,24,Master's,Low
S1003,Omar,Williams,student3@university.com,Female,24,High School,High
S1004,John,Smith,student4@university.com,Female,23,High School,High
S1005,Liam,Brown,student5@university.com,Male,21,PhD,High
S1006,Ahmed,Jones,student6@university.com,Male,24,null,Low
S1007,Ahmed,Smith,student7@university.com,Male,19,None,Medium
S1008,Omar,Smith,student8@university.com,Female,21,Bachelor's,Low
S1009,Sara,Smith,student9@university.com,Female,22,None,Medium


In [0]:
%sql -- Criação e carregamento de tabela "Alunos"

DROP TABLE IF EXISTS default.ALUNOS;

CREATE TABLE default.ALUNOS
USING CSV
OPTIONS (
  path "dbfs:/FileStore/tables/2_PRATA/ALUNOS.csv",
  header "true",
  inferSchema "true"
);


###### 🗃️ Tabela Habitos

In [0]:
%sql -- Exemplo de dados --
SELECT * FROM csv_habitos LIMIT 10;

ID_Aluno,Study_Hours_per_Week,Extracurricular_Activities,Internet_Access_at_Home,Stress_Level (1-10),Sleep_Hours_per_Night
S1000,6.2,No,Yes,5,4.7
S1001,19.0,No,Yes,4,9.0
S1002,20.7,No,Yes,6,6.2
S1003,24.8,Yes,Yes,3,6.7
S1004,15.4,Yes,Yes,2,7.1
S1005,8.5,Yes,Yes,1,5.0
S1006,21.3,No,Yes,5,6.4
S1007,27.3,Yes,No,4,4.3
S1008,8.0,No,No,9,8.8
S1009,9.6,No,Yes,10,6.4


In [0]:
%sql -- Criação e carregamento de tabela "Habitos"
DROP TABLE IF EXISTS default.HABITOS;

CREATE TABLE default.HABITOS
USING CSV
OPTIONS (
  path "dbfs:/FileStore/tables/2_PRATA/HABITOS.csv",
  header "true",
  inferSchema "true"
);

🗃️ Tabela Disciplinas

In [0]:
%sql -- Exemplo de dados --
SELECT * FROM csv_disciplinas LIMIT 10;


Nome_Disciplina,ID_Disciplina
Engineering,0
Mathematics,1
Business,2
CS,3


In [0]:
%sql -- Criação e carregamento de tabela "Disciplinas" 
DROP TABLE IF EXISTS default.DISCIPLINAS;

CREATE TABLE default.DISCIPLINAS
USING CSV
OPTIONS (
  path "dbfs:/FileStore/tables/2_PRATA/DISCIPLINAS.csv",
  header "true",
  inferSchema "true"
);

🗃️ Fato Desempenho

In [0]:
%sql -- Exemplo de dados --
SELECT * FROM csv_fato_desempenho LIMIT 10;


ID_Aluno,ID_Disciplina,Presenca,Nota_Midterm,Nota_Final,Nota_Projetos,Nota_Total,Conceito
S1000,Engineering,52.29,55.03,57.82,85.9,56.09,F
S1001,Engineering,97.27,97.23,45.8,55.65,50.64,A
S1002,Business,57.19,67.05,93.68,73.79,70.3,D
S1003,Mathematics,95.15,47.79,80.63,92.12,61.63,A
S1004,CS,54.18,46.59,78.89,68.42,66.13,F
S1005,Engineering,null,78.85,43.53,67.29,62.08,B
S1006,Business,57.6,66.26,89.07,93.65,83.21,F
S1007,Engineering,51.91,45.67,73.96,93.24,81.93,F
S1008,CS,85.97,84.42,90.87,94.01,95.62,A
S1009,Engineering,64.01,87.96,98.47,78.6,84.99,A


In [0]:
%sql -- Criação e carregamento da fato_desempenho
DROP TABLE IF EXISTS default.FATO_DESEMPENHO;

CREATE TABLE default.FATO_DESEMPENHO
USING CSV
OPTIONS (
  path "dbfs:/FileStore/tables/2_PRATA/FATO_DESEMPENHO.csv",
  header "true",
  inferSchema "true"
);

#### 🔎 Análise Exploratória dos dados

###### 🔎 Apresentar as tabelas e visualizações do modelo

In [0]:
%sql
SHOW TABLES IN default;


database,tableName,isTemporary
default,alunos,false
default,disciplinas,false
default,fato_desempenho,false
default,habitos,false
,csv_alunos,true
,csv_disciplinas,true
,csv_fato_desempenho,true
,csv_habitos,true


🧮 Quantidade de Registros

In [0]:
%sql
SELECT 'alunos' AS tabela, COUNT(*) AS total FROM default.alunos
UNION ALL
SELECT 'disciplinas', COUNT(*) FROM default.disciplinas
UNION ALL
SELECT 'fato_desempenho', COUNT(*) FROM default.fato_desempenho
UNION ALL
SELECT 'habitos', COUNT(*) FROM default.habitos;


tabela,total
alunos,5000
disciplinas,4
fato_desempenho,5000
habitos,5000


In [0]:
%sql
SELECT 
    'alunos' AS tabela, 
    COUNT(*) AS total, 
    SUM(CASE WHEN ID_Aluno IS NULL THEN 1 ELSE 0 END) AS ID_Aluno_nulos,
    SUM(CASE WHEN First_Name IS NULL THEN 1 ELSE 0 END) AS First_Name_nulos
FROM default.alunos;


tabela,total,ID_Aluno_nulos,First_Name_nulos
alunos,5000,0,0


📊 Estatísticas descritivas (Exemplos)

In [0]:
%sql
SELECT 
    MIN(Age) AS idade_minima,
    MAX(Age) AS idade_maxima,
    AVG(Age) AS idade_media
FROM default.alunos;


idade_minima,idade_maxima,idade_media
18,24,21.0484


In [0]:
%sql
SELECT Age, COUNT(*) AS quantidade FROM default.alunos GROUP BY Age ORDER BY Age;


Age,quantidade
18,682
19,705
20,671
21,753
22,732
23,734
24,723


In [0]:
# histogram dos dados: distribuição de idade e distribuição de notas. 

import importlib.util
import subprocess

# Verifica se a biblioteca Plotly está instalada, caso contrário, instala

package_name = "plotly"

if importlib.util.find_spec(package_name) is None:
    print(f"{package_name} não encontrado. Instalando...")
    subprocess.run(["pip", "install", package_name])
else:
    print(f"{package_name} OK!")

#------------------------------

import plotly.express as px

# Verificar se 'Age' existe na tabela alunos
df_check = spark.sql("SHOW COLUMNS IN default.alunos").toPandas()
if 'Age' not in df_check['col_name'].values:
    raise ValueError("A coluna 'Age' não existe na tabela 'alunos'. Verifique os nomes das colunas.")

# Criar DataFrame unindo as tabelas (Age e Nota_Final)
query = """
SELECT A.Age, F.Nota_Final 
FROM default.alunos A
JOIN default.fato_desempenho F 
ON A.ID_Aluno = F.ID_Aluno
WHERE A.Age IS NOT NULL AND F.Nota_Final IS NOT NULL
"""

df = spark.sql(query).toPandas()

# Criar histograma da distribuição de idades
fig1 = px.histogram(df, x="Age", nbins=20, title="Distribuição de Idades - Alunos")
fig1.show()

# Criar histograma da distribuição de notas finais
fig2 = px.histogram(df, x="Nota_Final", nbins=20, title="Distribuição das Notas Finais")
fig2.show()

plotly OK!


In [0]:
# Importar bibliotecas necessárias
import plotly.express as px
import pandas as pd

# Listar tabelas para análise
tables = ["alunos", "disciplinas", "fato_desempenho", "habitos"]

# Criar dicionário para armazenar os DataFrames dos gráficos
df_list = {}

for table in tables:
    # Ler os dados
    df = spark.sql(f"SELECT * FROM default.{table}").toPandas()

    # Contar valores válidos e nulos
    total_values = df.size  # Total de células (linhas x colunas)
    null_values = df.isnull().sum().sum()  # Total de valores nulos
    valid_values = total_values - null_values  # Total de valores válidos

    # Criar DataFrame específico para essa tabela
    df_list[table] = pd.DataFrame({
        "Status": ["Válidos", "Nulos"],
        "Quantidade": [valid_values, null_values]
    })

# Criar gráficos individuais com rótulos nos dados
for table, df_plot in df_list.items():
    fig = px.bar(df_plot, x="Status", y="Quantidade", title=f"Quantidade de Dados - {table}", 
                 barmode="group", text="Quantidade")
    
    # Ajustar layout para exibir os rótulos corretamente
    fig.update_traces(textposition="outside")
    
    # Mostrar gráfico
    fig.show()


### Entrega 📫

#### ❗ Problema
Parte dos alunos apresenta baixo desempenho acadêmico, e as causas nem sempre são claras, dificultando ações preventivas e direcionadas.

In [0]:
%sql
-- Distribuição de desempenho por conceito

SELECT 
  Conceito,
  COUNT(*) AS Quantidade_de_Registros
FROM 
  default.fato_desempenho
GROUP BY 
  Conceito
ORDER BY 
  Quantidade_de_Registros DESC;


Conceito,Quantidade_de_Registros
A,1495
B,978
D,889
F,844
C,794


In [0]:
df_conceitos = spark.sql("""
SELECT 
  Conceito,
  COUNT(*) AS Quantidade_de_Registros
FROM 
  default.fato_desempenho
GROUP BY 
  Conceito
""").toPandas()

# Calcular percentual
df_conceitos['Percentual'] = (df_conceitos['Quantidade_de_Registros'] / df_conceitos['Quantidade_de_Registros'].sum()) * 100

# Ordenar pela ordem lógica dos conceitos
ordem_conceitos = ['A', 'B', 'C', 'D', 'F']
df_conceitos['Conceito'] = pd.Categorical(df_conceitos['Conceito'], categories=ordem_conceitos, ordered=True)
df_conceitos = df_conceitos.sort_values('Conceito')

# Criar gráfico
import plotly.express as px

fig = px.bar(
    df_conceitos,
    x='Percentual',
    y='Conceito',
    orientation='h',
    text=df_conceitos['Percentual'].apply(lambda x: f'{x:.1f}%'),
    title='Percentual de Alunos por Conceito de Desempenho',
    labels={'Percentual': 'Percentual (%)', 'Conceito': 'Conceito'}
)

fig.update_traces(marker_color='teal', textposition='outside')
fig.update_layout(yaxis=dict(categoryorder='array', categoryarray=ordem_conceitos[::-1]))

fig.show()

#---------------------


# Extrair os percentuais de D e F
pct_D = df_conceitos.loc[df_conceitos['Conceito'] == 'D', 'Percentual'].values[0]
pct_F = df_conceitos.loc[df_conceitos['Conceito'] == 'F', 'Percentual'].values[0]
pct_total = pct_D + pct_F

# Construir texto markdown dinâmico
texto_md = f"""
💬 O gráfico demonstra que {pct_D:.1f}% dos alunos têm nota D e {pct_F:.1f}% dos estudantes têm nota F.  
Em síntese, {pct_total:.1f}% registram **baixo desempenho**.
"""

# Exibir markdown no notebook
displayHTML(f"<div style='font-size:16px'>{texto_md}</div>")


💬 O gráfico demonstra que 17.8% dos alunos têm nota D e 16.9% dos estudantes têm nota F. 
Em síntese, 34.7% registram **baixo desempenho**.

#### 🎯 Objetivo
Identificar os principais fatores comportamentais e socioeconômicos que influenciam negativamente o desempenho acadêmico dos alunos.

In [0]:
%sql
-- Criar no banco de dados a tabela de alunos unificados para analisarmos todas as caracteristicas dos alunos.

CREATE OR REPLACE TABLE default.alunos_unificados AS
SELECT 
    f.ID_Aluno,
    f.ID_Disciplina,
    f.Presenca,
    f.Nota_Midterm,
    f.Nota_Final,
    f.Nota_Projetos,
    f.Nota_Total,
    f.Conceito,
    a.Gender,
    a.Age,
    a.Parent_Education_Level,
    a.Family_Income_Level,
    h.Study_Hours_per_Week,
    h.Extracurricular_Activities,
    h.Internet_Access_at_Home,
    h.`Stress_Level (1-10)` AS Stress_Level,
    h.Sleep_Hours_per_Night
FROM 
    default.fato_desempenho f
LEFT JOIN 
    default.alunos a ON f.ID_Aluno = a.ID_Aluno
LEFT JOIN 
    default.habitos h ON f.ID_Aluno = h.ID_Aluno;


num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Criando uma view para analisar alunos com baixo desempenho.

CREATE OR REPLACE TABLE default.alunos_baixo_desempenho AS
SELECT *
FROM default.alunos_unificados
WHERE Conceito IN ('F');


num_affected_rows,num_inserted_rows


#### 💡 Solução
Integrar e analisar os dados de desempenho, hábitos e perfil socioeconômico dos alunos para encontrar padrões que indiquem risco de baixo desempenho, auxiliando na definição de estratégias de suporte mais eficazes.

###### 👣 Passo 1: Classificar os alunos por grupo de desempenho
###### 👣 Passo 2: Comparar fatores entre os grupos
###### 👣 Passo 3: Responder perguntas chave

In [0]:
%sql -- Coluna Desempenho
CREATE OR REPLACE VIEW default.alunos_com_desempenho AS
SELECT *,
  CASE 
    WHEN Conceito IN ('A', 'B') THEN 'Alto'
    WHEN Conceito IN ('F') THEN 'Baixo'
    ELSE 'Medio'
  END AS Desempenho
FROM default.alunos_unificados;


In [0]:
%sql
SELECT 
  Desempenho,
  ROUND(AVG(Study_Hours_per_Week), 2) AS Media_Horas_Estudo,
  COUNT(*) AS Total_Alunos
FROM default.alunos_com_desempenho
GROUP BY Desempenho
ORDER BY 
  CASE Desempenho
    WHEN 'Alto' THEN 1
    WHEN 'Medio' THEN 2
    WHEN 'Baixo' THEN 3
    ELSE 4
  END;


Desempenho,Media_Horas_Estudo,Total_Alunos
Alto,17.71,2473
Medio,17.44,1683
Baixo,17.95,844


In [0]:
spark.table("default.alunos_unificados") \
     .coalesce(1) \
     .write.mode('overwrite') \
     .option("header", True) \
     .csv("dbfs:/FileStore/tables/alunos_unificados")

arquivo_origem = dbutils.fs.ls("dbfs:/FileStore/tables/alunos_unificados/")[2].path
arquivo_destino = "dbfs:/FileStore/tables/alunos_unificados.csv"

dbutils.fs.mv(arquivo_origem, arquivo_destino, True)

display(arquivo_destino)

'dbfs:/FileStore/tables/alunos_unificados.csv'